In [ ]:
from pathlib import Path

import Metashape

from result import Ok, Err, Result

from benthoscan.registration import PointCloud, PointCloudLoader
from benthoscan.backends import metashape as backend
from benthoscan.utils.log import logger

In [ ]:
PATHS: dict[str, Path] = {
    "DOCUMENT_IN": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz"
    ),
    "DOCUMENT_OUT": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects_test/r23685bc_working_version_saved.psz"
    ),
    "CACHE": Path("/home/martin/dev/benthoscan/.cache/"),
}

result: Result[str, str] = backend.load_project(PATHS.get("DOCUMENT_IN"))
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

backend.log_internal_data()

### Backend playground

In [ ]:
document: Metashape.Document = backend.context._backend_data.get("document")

chunk: Metashape.Chunk = document.chunks[0]
valid_cameras: list[Metashape.Camera] = [
    camera for camera in chunk.cameras if camera.transform
]

camera: Metashape.Camera = valid_cameras[0]

logger.info(chunk)
logger.info(camera)
logger.info(camera.sensor)
logger.info(chunk.model)

range_map: Metashape.Image = camera.chunk.model.renderDepth(
    camera.transform, camera.sensor.calibration, add_alpha=False
)
normal_map: Metashape.Image = camera.chunk.model.renderNormalMap(
    camera.transform, camera.sensor.calibration, add_alpha=False
)

for image in [range_map, normal_map]:
    logger.info(f"Channels:      {image.channels}")
    logger.info(f"Channel count: {image.cn}")
    logger.info(f"Data type:     {image.data_type}")
    logger.info("")

### Export dense models from the backend

In [ ]:
export_point_clouds: bool = False

if export_point_clouds:
    PointCloudLoaders = dict[int, PointCloudLoader]

    result: Result[PointCloudLoaders, str] = backend.request_dense_models(
        PATHS["CACHE"], overwrite=True
    )

### Test stereo depth estimation on camera pairs from Metashape chunks

In [ ]:
document: Metashape.Document = backend.context._backend_data.get("document")

target_labels: list[str] = ["r23685bc_20100605_021022"]
target_chunks: list[Metashape.Chunk] = [
    chunk for chunk in document.chunks if chunk.label in target_labels
]


output_root: Path = Path("/data/kingston_snv_01/rendered_range_maps")